In [ ]:
!pip install cmake
!pip install dlib    # (c++) visual studio 17 com (c++) visual studio build tools
!pip install face_recognition

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
import os
os.chdir("/content/drive/MyDrive/myfiles/Face Recgonition")

In [ ]:
# import the necessary packages
# import tensorflow
from imutils import paths
import face_recognition
import argparse
import pickle
import cv2
import os
import time
import imutils
from tqdm import tqdm_notebook as tqdm

In [ ]:
# variables

DATASET = "GOT_DATASET"
ENCODINGS = "encodings/GOT_GPU.pickle"

# The CNN method is more accurate but slower.
# HOG is faster but less accurate.
METHOD = "cnn"

In [ ]:
# grab the paths to the input images in our dataset

print("[INFO] quantifying faces...")
imagePaths = list(paths.list_images(DATASET))

print(imagePaths[:5]) # display first five paths

In [ ]:
print("total images - {}".format(len(imagePaths)))

In [ ]:
imagePaths[0].split(os.path.sep)

In [ ]:
# extract name from the image path
imagePaths[0].split(os.path.sep)[-2]

'Tyrion Lannister'

For each iteration of the loop, we’re going to detect a face (or possibly multiple faces and assume that it is the same person in multiple locations of the image — this assumption may or may not hold true in your own images so be careful here).

In [ ]:
# initialize the list of known encodings and known names
knownEncodings = []
knownNames = []
no_face = []


# progress bar
with tqdm(total=len(imagePaths)) as pbar:
    # loop over the image paths
    for (i, imagePath) in enumerate(imagePaths):

        # extract the person name from the image path
        print("[INFO] processing image {}/{}".format(i + 1, len(imagePaths)))
        name = imagePath.split(os.path.sep)[-2]

        try:

            # load the input image and convert it from BGR (OpenCV ordering)
            # to dlib ordering (RGB)
            image = cv2.imread(imagePath)
            rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            rgb = cv2.resize(rgb, None, fx=0.5, fy=0.5)
        except:
            print("[INFO] skipping the image..{}".format(imagePath))
            continue

        # detect the (x, y)-coordinates of the bounding boxes
        # corresponding to each face in the input image
        boxes = face_recognition.face_locations(rgb, model=METHOD)

        print("[INFO] name: {}  | path: {}".format(name, imagePath))

        # compute the facial embedding for the face
        encodings = face_recognition.face_encodings(rgb, boxes)

        if len(encodings) == 0:
            no_face.append(imagePath)

        print("[INFO] face: {}  |  encodings: {}".format(len(boxes),len(encodings)))

        # loop over the encodings
        for encoding in encodings:

            # add each encoding + name to our set of known names and
            # encodings

            knownEncodings.append(encoding)
            knownNames.append(name)
    pbar.update(1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/312 [00:00<?, ?it/s]

[INFO] processing image 1/312
[INFO] name: Tyrion Lannister  | path: GOT_DATASET/Tyrion Lannister/TyrionLannister.jpg
[INFO] face: 1  |  encodings: 1
[INFO] processing image 2/312
[INFO] name: Tyrion Lannister  | path: GOT_DATASET/Tyrion Lannister/tyrion-lannister-759.jpg
[INFO] face: 1  |  encodings: 1
[INFO] processing image 3/312
[INFO] name: Tyrion Lannister  | path: GOT_DATASET/Tyrion Lannister/tyrion.jpg
[INFO] face: 1  |  encodings: 1
[INFO] processing image 4/312
[INFO] name: Tyrion Lannister  | path: GOT_DATASET/Tyrion Lannister/fde1b5b843f0d3e2937164a02297efdf.jpg
[INFO] face: 1  |  encodings: 1
[INFO] processing image 5/312
[INFO] name: Tyrion Lannister  | path: GOT_DATASET/Tyrion Lannister/0613420595ca160f3f9f841cb744517f.jpg
[INFO] face: 1  |  encodings: 1
[INFO] processing image 6/312
[INFO] name: Tyrion Lannister  | path: GOT_DATASET/Tyrion Lannister/1OtG2bOqXGBQYWqw7Tef_Tg.jpeg
[INFO] face: 1  |  encodings: 1
[INFO] processing image 7/312
[INFO] name: Tyrion Lannister  

In [ ]:
knownEncodings

In [ ]:
print(len(knownEncodings))
print(len(knownNames))
print(len(no_face))

275
275
66


In [ ]:
print("Known Encodings :- ".format(knownEncodings))
print()
print()
print("Known Names :- {}".format(knownNames))

Known Encodings :- 


Known Names :- ['Tyrion Lannister', 'Tyrion Lannister', 'Tyrion Lannister', 'Tyrion Lannister', 'Tyrion Lannister', 'Tyrion Lannister', 'Tyrion Lannister', 'Tyrion Lannister', 'Tyrion Lannister', 'Tyrion Lannister', 'Tyrion Lannister', 'Tyrion Lannister', 'Tyrion Lannister', 'Tyrion Lannister', 'Tyrion Lannister', 'Tyrion Lannister', 'Tyrion Lannister', 'Tyrion Lannister', 'Tyrion Lannister', 'Tyrion Lannister', 'Tyrion Lannister', 'Tyrion Lannister', 'Tyrion Lannister', 'Tyrion Lannister', 'Tyrion Lannister', 'Tyrion Lannister', 'Tyrion Lannister', 'Tyrion Lannister', 'Tyrion Lannister', 'Tyrion Lannister', 'Tyrion Lannister', 'Tyrion Lannister', 'Tyrion Lannister', 'Tyrion Lannister', 'Tyrion Lannister', 'Tyrion Lannister', 'Tyrion Lannister', 'Tyrion Lannister', 'Tyrion Lannister', 'Tyrion Lannister', 'New folder', 'New folder', 'New folder', 'New folder', 'New folder', 'New folder', 'New folder', 'New folder', 'New folder', 'New folder', 'New folder', 'New fol

In [ ]:
# dump the facial encodings + names to disk
print("[INFO] serializing encodings...")
data = {"encodings": knownEncodings, "names": knownNames}
f = open(ENCODINGS, "wb")
f.write(pickle.dumps(data))
f.close()

[INFO] serializing encodings...


## recognize faces in image

In [ ]:
# load the known faces and embeddings
print("[INFO] loading encodings...")
data = pickle.loads(open(ENCODINGS, "rb").read())

[INFO] loading encodings...


In [ ]:
TEST_IMAGE = "samples/example_01.png"
ENCODINGS = "encodings/jurassic_park.pickle"

In [ ]:
# load the input image and convert it from BGR to RGB
image = cv2.imread(TEST_IMAGE)

In [ ]:
image

In [ ]:
rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

In [ ]:
# detect the (x, y)-coordinates of the bounding boxes corresponding
# to each face in the input image, then compute the facial embeddings
# for each face

print("[INFO] recognizing faces...")
boxes = face_recognition.face_locations(rgb, model=METHOD)

[INFO] recognizing faces...


In [ ]:
boxes

[(56, 474, 163, 366), (80, 175, 187, 67)]

In [ ]:
encodings = face_recognition.face_encodings(rgb, boxes)

In [ ]:
names = []

# loop over the facial embeddings
for encoding in encodings:
    # attempt to match each face in the input image to our known
    # encodings
    matches = face_recognition.compare_faces(data["encodings"], encoding)
    name = "Unknown"

    # check to see if we have found a match
    if True in matches:
        # find the indexes of all matched faces then initialize a
        # dictionary to count the total number of times each face
        # was matched
        matchedIdxs = [i for (i, b) in enumerate(matches) if b]
        counts = {}

        # loop over the matched indexes and maintain a count for
        # each recognized face face
        for i in matchedIdxs:
            name = data["names"][i]
            counts[name] = counts.get(name, 0) + 1

        # determine the recognized face with the largest number of
        # votes (note: in the event of an unlikely tie Python will
        # select first entry in the dictionary)
        name = max(counts, key=counts.get)

    # update the list of names
    names.append(name)


In [ ]:
# loop over the recognized faces
for ((top, right, bottom, left), name) in zip(boxes, names):

    # draw the predicted face name on the image
    cv2.rectangle(image, (left, top), (right, bottom), (0, 255, 0), 2)
    y = top - 15 if top - 15 > 15 else top + 15
    cv2.putText(image, name, (left, y), cv2.FONT_HERSHEY_SIMPLEX,
        0.75, (0, 255, 0), 2)


# image = cv2.resize(image, None, fx=0.3, fy=0.3)

# show the output image
cv2.imshow("Image", image)
cv2.waitKey()
cv2.destroyAllWindows()

In [ ]:
cv2.destroyAllWindows()

## Recognize faces in videos

In [ ]:
TEST_VIDEO = "samples/vid1.mp4"
ENCODINGS = "encodings/GOT_GPU.pickle"
METHOD = "cnn"
OUTPUT = "samples/GOT_GPU_OUT_vid2.mp4"
DISPLAY = 0

In [ ]:

# initialize the pointer to the video file and the video writer
print("[INFO] processing video...")
stream = cv2.VideoCapture(TEST_VIDEO)
writer = None

time.sleep(2)

tot_frames = int(stream.get(cv2.CAP_PROP_FRAME_COUNT))
# loop over frames from the video file stream

with tqdm(total=tot_frames) as pbar:
    while True:
        # grab the next frame
        (grabbed, frame) = stream.read()

        # if the frame was not grabbed, then we have reached the
        # end of the stream
        if not grabbed:
            break

        # convert the input frame from BGR to RGB then resize it to have
        # a width of 750px (to speedup processing)
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        rgb = imutils.resize(frame, width=360)
        r = frame.shape[1] / float(rgb.shape[1])

        # detect the (x, y)-coordinates of the bounding boxes
        # corresponding to each face in the input frame, then compute
        # the facial embeddings for each face
        boxes = face_recognition.face_locations(rgb, model=METHOD)
        encodings = face_recognition.face_encodings(rgb, boxes)
        names = []

        # loop over the facial embeddings
        for encoding in encodings:
            # attempt to match each face in the input image to our known
            # encodings
            matches = face_recognition.compare_faces(data["encodings"], encoding)
            name = "Unknown"

            # check to see if we have found a match
            if True in matches:
                # find the indexes of all matched faces then initialize a
                # dictionary to count the total number of times each face
                # was matched
                matchedIdxs = [i for (i, b) in enumerate(matches) if b]
                counts = {}

                # loop over the matched indexes and maintain a count for
                # each recognized face face
                for i in matchedIdxs:
                    name = data["names"][i]
                    counts[name] = counts.get(name, 0) + 1

                # determine the recognized face with the largest number
                # of votes (note: in the event of an unlikely tie Python
                # will select first entry in the dictionary)
                name = max(counts, key=counts.get)

            # update the list of names
            names.append(name)

        # loop over the recognized faces
        for ((top, right, bottom, left), name) in zip(boxes, names):
            # rescale the face coordinates
            top = int(top * r)
            right = int(right * r)
            bottom = int(bottom * r)
            left = int(left * r)

            # draw the predicted face name on the image
            cv2.rectangle(frame, (left, top), (right, bottom),
                (0, 255, 0), 2)
            y = top - 15 if top - 15 > 15 else top + 15
            cv2.putText(frame, name, (left, y), cv2.FONT_HERSHEY_SIMPLEX,
                0.75, (0, 255, 0), 2)

        # if the video writer is None *AND* we are supposed to write
        # the output video to disk initialize the writer
        if writer is None and OUTPUT is not None:
            fourcc = cv2.VideoWriter_fourcc(*"MJPG")
            writer = cv2.VideoWriter(OUTPUT, fourcc, 24,
                (frame.shape[1], frame.shape[0]), True)

        # if the writer is not None, write the frame with recognized
        # faces t odisk
        if writer is not None:
            writer.write(frame)

        # check to see if we are supposed to display the output frame to
        # the screen
        if DISPLAY > 0:
            cv2.imshow("Frame", frame)
            key = cv2.waitKey(1) & 0xFF

            # if the `q` key was pressed, break from the loop
            if key == ord("q"):
                break

        pbar.update(1)

# close the video file pointers
stream.release()

# check to see if the video writer point needs to be released
if writer is not None:
    writer.release()
cv2.waitKey()
cv2.destroyAllWindows()

[INFO] processing video...


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if sys.path[0] == '':


  0%|          | 0/3088 [00:00<?, ?it/s]

In [ ]:
TEST_VIDEO = "samples/vid360p.mp4"
ENCODINGS = "encodings/GOT.pickle"
METHOD = "hog"
OUTPUT = None # "samples/out_vid.mp4"
DISPLAY = 1

In [ ]:

# initialize the pointer to the video file and the video writer
print("[INFO] processing video...")
stream = cv2.VideoCapture(TEST_VIDEO)
writer = None

time.sleep(2)

# loop over frames from the video file stream
while True:
    # grab the next frame
    (grabbed, frame) = stream.read()

    # if the frame was not grabbed, then we have reached the
    # end of the stream
    if not grabbed:
        break

    # convert the input frame from BGR to RGB then resize it to have
    # a width of 750px (to speedup processing)
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    rgb = cv2.resize(rgb, None, fx=0.3, fy=0.3)




    # detect the (x, y)-coordinates of the bounding boxes
    # corresponding to each face in the input frame, then compute
    # the facial embeddings for each face
    boxes = face_recognition.face_locations(rgb, model=METHOD)
    encodings = face_recognition.face_encodings(rgb, boxes)
    names = []

    # loop over the facial embeddings
    for encoding in encodings:
        # attempt to match each face in the input image to our known
        # encodings
        matches = face_recognition.compare_faces(data["encodings"], encoding)
        name = "Unknown"

        # check to see if we have found a match
        if True in matches:
            # find the indexes of all matched faces then initialize a
            # dictionary to count the total number of times each face
            # was matched
            matchedIdxs = [i for (i, b) in enumerate(matches) if b]
            counts = {}

            # loop over the matched indexes and maintain a count for
            # each recognized face face
            for i in matchedIdxs:
                name = data["names"][i]
                counts[name] = counts.get(name, 0) + 1

            # determine the recognized face with the largest number
            # of votes (note: in the event of an unlikely tie Python
            # will select first entry in the dictionary)
            name = max(counts, key=counts.get)

        # update the list of names
        names.append(name)

    # loop over the recognized faces
    for ((top, right, bottom, left), name) in zip(boxes, names):


        # draw the predicted face name on the image
        cv2.rectangle(frame, (left, top), (right, bottom),
            (0, 255, 0), 2)
        y = top - 15 if top - 15 > 15 else top + 15
        cv2.putText(frame, name, (left, y), cv2.FONT_HERSHEY_SIMPLEX,
            0.75, (0, 255, 0), 2)



    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF

    # if the `q` key was pressed, break from the loop
    if key == ord("q"):
        break

# close the video file pointers
stream.release()

cv2.waitKey()
cv2.destroyAllWindows()

[INFO] processing video...
